In [1]:
from final_utils import *
from Sufflenet_CNN_FC import *

In [2]:
root_path = generate_root_path()
SC_CE_KLD = tf.keras.losses.SparseCategoricalCrossentropy()

### DataLoader and other nesseccary functions

In [3]:
def binary_accuracy(y_true, y_pred): 
    return ((tf.math.argmax(y_pred, axis=1)) == tf.cast(y_true, dtype=tf.dtypes.int64)).numpy().mean()

def lr_exp_decay(epoch, lr):
    k = 0.04
    return lr * np.exp(-k*epoch)

def data_loader_connected(data, labels, resize_func=None, size=None):
    #big pictures require lots of computational data, so we resize them
    images = []
    count = 0
    for i,path in enumerate(data):
        try:
            img = Image.open(path).convert('RGB')
            img_tensor = tf.keras.utils.img_to_array(img)
            img_tensor = img_tensor / 255
            if resize_func:
                img_tensor = resize_func(img_tensor, size=size)
            images.append(img_tensor)
        except:
            print('Skip')
            labels = np.delete(labels, i - count)
            count += 1
            
    images = tf.stack(images)
    
    return images, labels

### Trainer

In [4]:
def trainer(model, data, data_val, batch_size=16, epochs=20, learning_rate=0.003, save_to=None):
    X_train, y_train = data
    X_train, y_train = data_loader_connected(X_train, y_train)
    print(X_train.shape)
    print(y_train.shape)
    
    model.compile(loss=SC_CE_KLD,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, 
                                                  epsilon=1e-07, decay=0, amsgrad=False))
    
    schedule = tf.keras.callbacks.LearningRateScheduler(lr_exp_decay, verbose=0)
    callbacks_list = [schedule]

    history = model.fit(
                    X_train, 
                    y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data = data_val,
                    callbacks=callbacks_list,
                    verbose=1
                )
    
    print('Done')
    return history

### Loading data

In [5]:
main_path = main_path=f'{root_path}Data/splitted/train/'
paths_bad = []
paths_good = []
    
for i in range(7):
    alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
    bad_data = json.load(alm_train_bad)
    
    for data in bad_data:
        path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
        paths_bad.append(path_to_img)
        
alm_train_good = open(f'{main_path}/data_good1.json')
good_data = json.load(alm_train_good)
for data in good_data:
    path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
    paths_good.append(path_to_img)
    
for i in range(7):
    paths_bad[i] = np.squeeze(np.array(paths_bad[i]))
paths_good = np.squeeze(np.array(paths_good))
   
# Generating static validation data
paths_bad, paths_bad_val = extract_static_val_data(paths_bad, perc = 0.0009) #original - 0.017
paths_good, paths_good_val = extract_static_val_data(paths_good, perc = 0.005) #original - 0.11

paths_bad = np.array(paths_bad)
paths_good = np.array(paths_good)
paths_bad_val = np.array(paths_bad_val)
paths_good_val = np.array(paths_good_val)
labels_good = np.ones(len(paths_good))
labels_bad = np.zeros(len(paths_bad))

In [6]:
full_data = np.concatenate((np.repeat(paths_good, 7), paths_bad))
full_labels = np.concatenate(((np.repeat(labels_good, 7), labels_bad)))   

#shuffling
idx = np.random.permutation(len(full_data))
full_data = full_data[idx]
print(full_data.shape)
print(full_labels.shape)

(152546,)
(152546,)


In [7]:
#Splitting data 
split_factor = 8
splitted_data = []
splitted_labels = []

global_batches = int(full_data.shape[0] / split_factor)
for i in range(global_batches):
    batch_data = full_data[i*split_factor: (i+1)*split_factor]
    batch_labels = full_labels[i*split_factor: (i+1)*split_factor]
    
    splitted_labels.append(batch_labels)
    splitted_data.append(batch_data)
    

splitted_data[-1] = np.concatenate((splitted_data[-1], full_data[len(splitted_data)*split_factor:]))
splitted_labels[-1] = np.concatenate((splitted_labels[-1], full_labels[len(splitted_labels)*split_factor:]))

data = splitted_data, splitted_labels

In [8]:
#Loading validation data
paths_val = np.concatenate((paths_bad_val, paths_good_val ), axis=0 )
val_labels = np.concatenate((np.zeros(len(paths_bad_val)), np.ones(len(paths_good_val))))
X_val, y_val = data_loader_connected(paths_val, val_labels)

data_val = (X_val, y_val)

### Creating model and training

In [9]:
batch_size = 2
epochs = 10
learning_rate = 0.003

In [10]:
model = Connected()

In [11]:
# x = tf.random.uniform((1, 600, 600, 3))
# model.build([1, 600, 600, 3])
# model.predict(x, 1)

In [12]:
print('Training started')
for i in range(len(data)):
    batch_data = data[0][i], data[1][i]
    history = trainer(model, 
                      batch_data, 
                      data_val,
                      batch_size=batch_size,
                      epochs=epochs, 
                      learning_rate=learning_rate)
    model.save_weights(f'models/ShuffleNet_CNN_FC/Connected_on_600x600', save_format='tf')
    
    del batch_data

Training started
(8, 600, 600, 3)
(8,)
Epoch 1/10
4/4 [==============================] - ETA: 0s - loss: 1.0475

TypeError: in user code:

    File "C:\Users\CareAware\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "C:\Users\CareAware\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\CareAware\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "C:\Users\CareAware\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1499, in test_step
        y_pred = self(x, training=False)
    File "C:\Users\CareAware\.conda\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\CAREAW~1\AppData\Local\Temp\__autograph_generated_filef8m_38us.py", line 10, in tf__call
        x1 = ag__.conve

### Training on dummy data

In [ ]:
img_data = tf.random.uniform((64, 600, 600, 3))
labels = np.random.randint(0,1,(64, 16928))
data = img_data, labels

In [ ]:
img_data = tf.random.uniform((4, 600, 600, 3))
labels = np.random.randint(0,1,(4, 16928))
data_val = img_data, labels

In [ ]:
type(labels)

In [ ]:
labels = tf.convert_to_tensor(labels)

In [ ]:
type(img_data)

In [ ]:
history = trainer(model, 
                  data, 
                  weights_path,
                  data_val,
                  batch_size=batch_size,
                  epochs=epochs, 
                  learning_rate=learning_rate)